In [ ]:
from openai import OpenAI
import openai
import pandas as pd
import tiktoken
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = ""
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id

In [ ]:
import pandas as pd
import tiktoken

Here we are using Amazon's review dataset that aims to identify whether the sentiment of a product review is positive or negative.

In [ ]:
path = ""
df = pd.read_csv(path, index_col=0)

In [ ]:
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

In [ ]:
df.head(2)

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))

Wen are using GTE base model as the tokenizer, but the maximum context length is 768, which means there are certain responses that are too long at runtime. Therefore we are droping the indices here.

In [ ]:
embedding = []
indices_to_drop = []
import time
a = time.time()
for i in range(len(df)):
    text = df["combined"].iloc[i].replace("\n", " ")
    try:
        response = client.embeddings.create(input=[text], model="iic/nlp_gte_sentence-embedding_english-base")
        
    except openai.BadRequestError:
        indices_to_drop.append(i)
        continue
    indices_to_drop.append(response.data[0].embedding)
    #debug code
    if i%1000 == 0:
        print(time.time()-a)
        print(len(indices_to_drop))

In [ ]:
df["embedding"]=embedding

In [ ]:
path_to_save = ""
df.to_csv(path_to_save)